In [4]:
from dotenv import load_dotenv

if not load_dotenv("../config/.env"):
    raise Exception("Failed to load the .env file")

In [5]:


#制备Bing搜索
from langchain_community.tools import BingSearchRun
from langchain_community.utilities import BingSearchAPIWrapper
search_api = BingSearchAPIWrapper(k=3)
searchtool = BingSearchRun(api_wrapper=search_api)
searchtool.name = 'BingSearch'
searchtool.description ='This is Bing search tool. Useful for searching some real time info, such as news.'


#制备维基百科工具.定义name，description，JSON schema，the function to call, result return to user directly or not
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_result=1, doc_content_chars_max=100)
wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)
wikitool.name = 'Wikipedia'
wikitool.description ='A wrapper around Wikipedia. Useful for when you need to answer general question about definition and the description of people, place, facrts, history etc.'


# 制备Google搜索
# from langchain.utilities import SerpAPIWrapper
# from langchain_community.tools import GoogleSerperRun
# serp_api = SerpAPIWrapper()
# google_tool = GoogleSerperRun(api_wrapper=serp_api)
# google_tool.name = 'GoogleSearch'
# google_tool.description = 'This is Google search tool. Useful for searching some real time info, such as news.'



from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain.chains import LLMMathChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

llm_math_chain = LLMMathChain.from_llm(llm)

tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
    searchtool,
    wikitool,
]

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Adapted from https://smith.langchain.com/hub/hwchase17/openai-tools-agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [7]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"messages": [HumanMessage(content="I'm Nemo!")]})

In [9]:
agent_executor.invoke(
    {
        "messages": [
            HumanMessage(
                content="What is the current conservation status of the Great Barrier Reef?"
            )
        ],
    }
)

In [11]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "messages": [
            HumanMessage(content="I'm Nemo!"),
            AIMessage(content="Hello Nemo! How can I assist you today?"),
            HumanMessage(content="What is my name?"),
        ],
    }
)